# Tracking FII flows

Plots the Foreign Institutional Investment flows in/out of Indian capital markets.
Follows the documentation [here](https://github.com/shyams80/plutons/blob/master/docs-R/InvestmentFlowsIndia.ipynb)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
source("config.R")
source("goofy/plot.common.R")

#initialize
iflows <- InvestmentFlowsIndia()

In [ ]:
fiiDebt <- iflows$FiiCashMarket() %>%
    filter(SECURITY_TYPE == 'Debt') %>%
    mutate(Y=year(TIME_STAMP), M=month(TIME_STAMP)) %>%
    group_by(Y, M) %>%
    summarize(NET = sum(BUY_VALUE - SELL_VALUE)) %>%
    arrange(Y, M) %>%
    select(Y, M, NET) %>%
    as.data.frame()

print(head(fiiDebt))

### Debt flows

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)